# goals

the goal of this notebook is to come up with the best possible combination of all models in terms of training error.

then maybe ill try to xgb stuff but porbably not.

In [1]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc
from scipy.stats import rankdata

from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from xgboost import XGBClassifier, XGBRegressor

import keras
from keras import backend as K
from keras import optimizers
from keras.models import Model, load_model

from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16

from keras.applications.inception_v3 import preprocess_input as preprocess_input_incep_xcep
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet_vgg

from tensorflow.python.client import device_lib
device_lib.list_local_devices(), 'keras version: {}'.format(keras.__version__)

/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


([name: "/cpu:0"
  device_type: "CPU"
  memory_limit: 268435456
  locality {
  }
  incarnation: 14533971318638484554, name: "/gpu:0"
  device_type: "GPU"
  memory_limit: 11332668621
  locality {
    bus_id: 2
  }
  incarnation: 3369421686794756412
  physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:84:00.0"],
 'keras version: 2.0.6')

In [2]:
path = os.path.join('/scratch', 'yns207', 'data_invasive')
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

def delete_model(model, clear_session=True):
    '''removes model!
    '''
    del model
    gc.collect()
    if clear_session: K.clear_session()

def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

In [3]:
train_img_224, test_img_224 = read_imgs(224,224)
train_img_299, test_img_299 = read_imgs(299,299)
train_img_450, test_img_450 = read_imgs(450,450)
train_labels = train_set['invasive'].values

100%|██████████| 1531/1531 [00:53<00:00, 28.73it/s]


In [4]:
model_files = [
    'invasive_incepv3_aug10_kfolds_299x299_0.model',
    'invasive_incepv3_aug10_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_224x224_0.model',
    'invasive_xcep_aug11_kfolds_224x224_1.model',
    'invasive_xcep_aug11_kfolds_224x224_2.model',
    'invasive_xcep_aug11_kfolds_224x224_3.model',
    'invasive_xcep_aug11_kfolds_224x224_4.model',
    'invasive_xcep_aug11_kfolds_299x299_0.model',
    'invasive_xcep_aug11_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_2.model',
    'invasive_xcep_aug11_kfolds_299x299_3.model',
    'invasive_xcep_aug11_kfolds_299x299_4.model',
    'invasive_xcep_aug13_pseudo_224x224_0.model',
    'invasive_xcep_aug13_pseudo_224x224_1.model',
    'invasive_xcep_aug13_pseudo_224x224_2.model',
    'invasive_xcep_aug13_pseudo_224x224_3.model',
    'invasive_xcep_aug13_pseudo_224x224_4.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_0.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_3.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_4.model',
    'invasive_incepv3_aug11_kfolds_224x224_0.model',
    'invasive_incepv3_aug11_kfolds_224x224_1.model',
    'invasive_incepv3_aug11_kfolds_224x224_2.model',
    'invasive_incepv3_aug11_kfolds_224x224_3.model',
    'invasive_incepv3_aug11_kfolds_224x224_4.model',
    'invasive_incepv3_aug11_kfolds_299x299_0.model',
    'invasive_incepv3_aug11_kfolds_299x299_1.model',
    'invasive_incepv3_aug11_kfolds_299x299_2.model',
    'invasive_incepv3_aug11_kfolds_299x299_3.model',
    'invasive_incepv3_aug11_kfolds_299x299_4.model',
    'invasive_incepv3_aug11_kfolds_450x450_0.model',
    'invasive_incepv3_aug11_kfolds_450x450_1.model',
    'invasive_incepv3_aug11_kfolds_450x450_2.model',
    'invasive_incepv3_aug11_kfolds_450x450_3.model',
    'invasive_incepv3_aug11_kfolds_450x450_4.model',
    'invasive_incepv3_aug13_pseudo_224x224_0.model',
    'invasive_incepv3_aug13_pseudo_224x224_1.model',
    'invasive_incepv3_aug13_pseudo_224x224_2.model',
    'invasive_incepv3_aug13_pseudo_224x224_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug13_pseudo_299x299_0.model',
    'invasive_incepv3_aug13_pseudo_299x299_1.model',
    'invasive_incepv3_aug13_pseudo_299x299_2.model',
    'invasive_incepv3_aug13_pseudo_299x299_3.model',
    'invasive_incepv3_aug13_pseudo_299x299_4.model',
    'invasive_incepv3_aug13_pseudo_450x450_0.model',
    'invasive_incepv3_aug13_pseudo_450x450_1.model',
    'invasive_incepv3_aug13_pseudo_450x450_2.model',
    'invasive_incepv3_aug13_pseudo_450x450_3.model',
    'invasive_incepv3_aug13_pseudo_450x450_4.model',
]

preds_df_train = pd.DataFrame([], columns=['name'])
preds_df_train['name'] = train_labels

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_inceptionv3(train_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_inceptionv3(train_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_inceptionv3(train_img_450.astype(np.float32))
    preds_df_train[model_name] = pd.Series(model.predict(proc_img).flatten())
    delete_model(model)
preds_df_train.head()

invasive_incepv3_aug10_kfolds_299x299_0.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


invasive_incepv3_aug10_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_224x224_0.model
invasive_xcep_aug11_kfolds_224x224_1.model
invasive_xcep_aug11_kfolds_224x224_2.model
invasive_xcep_aug11_kfolds_224x224_3.model
invasive_xcep_aug11_kfolds_224x224_4.model
invasive_xcep_aug11_kfolds_299x299_0.model
invasive_xcep_aug11_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_299x299_2.model
invasive_xcep_aug11_kfolds_299x299_3.model
invasive_xcep_aug11_kfolds_299x299_4.model
invasive_xcep_aug13_pseudo_224x224_0.model
invasive_xcep_aug13_pseudo_224x224_1.model
invasive_xcep_aug13_pseudo_224x224_2.model
invasive_xcep_aug13_pseudo_224x224_3.model
invasive_xcep_aug13_pseudo_224x224_4.model
invasive_xcep_aug14_pseudo_ft_299x299_0.model
invasive_xcep_aug14_pseudo_ft_299x299_1.model
invasive_xcep_aug14_pseudo_ft_299x299_2.model
invasive_xcep_aug14_pseudo_ft_299x299_3.model
invasive_xcep_aug14_pseudo_ft_299x299_4.model
invasive_incepv3_aug11_kfolds_224x224_0.model
invasive_incepv3_aug11_kfolds_224

,name,invasive_incepv3_aug10_kfolds_299x299_0.model,invasive_incepv3_aug10_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_224x224_0.model,invasive_xcep_aug11_kfolds_224x224_1.model,invasive_xcep_aug11_kfolds_224x224_2.model,invasive_xcep_aug11_kfolds_224x224_3.model,invasive_xcep_aug11_kfolds_224x224_4.model,invasive_xcep_aug11_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_299x299_1.model,...,invasive_incepv3_aug13_pseudo_299x299_0.model,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_2.model,invasive_incepv3_aug13_pseudo_299x299_3.model,invasive_incepv3_aug13_pseudo_299x299_4.model,invasive_incepv3_aug13_pseudo_450x450_0.model,invasive_incepv3_aug13_pseudo_450x450_1.model,invasive_incepv3_aug13_pseudo_450x450_2.model,invasive_incepv3_aug13_pseudo_450x450_3.model,invasive_incepv3_aug13_pseudo_450x450_4.model
0,0,0.001253,0.011497,5.127164e-07,0.000289,0.000001,0.000003,0.000059,3.701978e-06,0.000081,...,0.030836,0.003738,0.023865,0.018126,0.004922,0.000463,0.042890,0.128249,0.049315,0.001428
1,0,0.002930,0.000088,3.061252e-02,0.004832,0.114906,0.373535,0.074218,2.146954e-02,0.001395,...,0.002686,0.024177,0.013861,0.038561,0.013948,0.017283,0.218183,0.099306,0.043599,0.132586
2,1,0.996199,0.999820,9.999363e-01,0.981235,0.990044,0.999787,0.999890,9.938554e-01,0.995244,...,0.992587,0.999223,0.999028,0.996538,0.997008,0.987502,0.999555,0.999991,0.997594,0.999424
3,0,0.000678,0.007591,5.869500e-05,0.000003,0.000002,0.000106,0.004041,9.471538e-07,0.000002,...,0.003795,0.002051,0.002503,0.009096,0.025521,0.006113,0.096105,0.116562,0.019150,0.008642
4,1,0.998524,1.000000,1.000000e+00,0.997346,1.000000,0.999998,0.999999,1.000000e+00,1.000000,...,0.996332,0.998412,0.999549,0.998529,0.999273,0.984641,0.979222,1.000000,0.999772,0.999913


In [26]:
keep_models = []
all_loss = 0
for col in preds_df_train.columns:
    train_preds = pd.DataFrame([], columns=['name', 'invasive'])
    if col == 'name':
        exclusion_list = []
    else:
        exclusion_list = [col]
        
    train_preds['invasive'] = preds_df_train[[c for c in preds_df_train.columns if not c in exclusion_list]].mean(axis=1)
    train_preds['name'] =  preds_df_train['name']
    
    # calculate the loss, roc, and accuracy 
    # without thes model
    loss_without = log_loss(train_labels, train_preds['invasive'])
    roc_without = roc_auc_score(train_labels, train_preds['invasive'])
    acc_without = accuracy_score(train_labels, np.around(train_preds['invasive']))
    
    print('without {}'.format(col))
    print(loss_without, roc_without, acc_without)
    
    if col == 'name':
        all_loss = loss_without
        
    print('is lower than all?')
    print(all_loss <= loss_without)
    
    # if loss withoht this model is higher than
    # the loss with all models keep the model
    # we want to aboid loss going higher.
    if (loss_without >= all_loss) and (col != 'name'):
        keep_models.append(col)

    print('\n')

without name
0.0292622780832 1.0 0.999564270153
is lower than all?
True


without invasive_incepv3_aug10_kfolds_299x299_0.model
0.0293598112138 1.0 0.999564270153
is lower than all?
True


without invasive_incepv3_aug10_kfolds_299x299_1.model
0.0292313532052 1.0 0.999564270153
is lower than all?
False


without invasive_xcep_aug11_kfolds_224x224_0.model
0.0292876349989 1.0 0.999564270153
is lower than all?
True


without invasive_xcep_aug11_kfolds_224x224_1.model
0.0289798750237 1.0 0.999128540305
is lower than all?
False


without invasive_xcep_aug11_kfolds_224x224_2.model
0.0292477115022 1.0 0.999564270153
is lower than all?
False


without invasive_xcep_aug11_kfolds_224x224_3.model
0.0292718248866 1.0 0.999564270153
is lower than all?
True


without invasive_xcep_aug11_kfolds_224x224_4.model
0.0291523804377 1.0 0.999128540305
is lower than all?
False


without invasive_xcep_aug11_kfolds_299x299_0.model
0.0294738112641 1.0 0.999564270153
is lower than all?
True


without invasive_xce

these are the models that we need to remove

In [27]:
keep_models, len(keep_models)

(['invasive_incepv3_aug10_kfolds_299x299_0.model',
  'invasive_xcep_aug11_kfolds_224x224_0.model',
  'invasive_xcep_aug11_kfolds_224x224_3.model',
  'invasive_xcep_aug11_kfolds_299x299_0.model',
  'invasive_xcep_aug11_kfolds_299x299_1.model',
  'invasive_xcep_aug11_kfolds_299x299_2.model',
  'invasive_xcep_aug11_kfolds_299x299_3.model',
  'invasive_xcep_aug11_kfolds_299x299_4.model',
  'invasive_xcep_aug13_pseudo_224x224_0.model',
  'invasive_xcep_aug13_pseudo_224x224_1.model',
  'invasive_xcep_aug13_pseudo_224x224_2.model',
  'invasive_xcep_aug13_pseudo_224x224_3.model',
  'invasive_xcep_aug13_pseudo_224x224_4.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_0.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_3.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_4.model',
  'invasive_incepv3_aug11_kfolds_224x224_1.model',
  'invasive_incepv3_aug11_kfolds_224x224_2.model',
  'invasive_ince

so ill keep these models. because taking them out makes the loss worse than the 'all model' loss. then ill see how thy perform together: 

In [28]:
train_preds = pd.DataFrame([], columns=['name', 'invasive'])

train_preds['invasive'] = preds_df_train[[c for c in preds_df_train.columns if c in keep_models]].mean(axis=1)
train_preds['name'] =  preds_df_train['name']

# calculate the loss, roc, and accuracy 
# without thes model
loss = log_loss(train_labels, train_preds['invasive'])
roc = roc_auc_score(train_labels, train_preds['invasive'])
acc = accuracy_score(train_labels, np.around(train_preds['invasive']))

print(loss, roc, acc)

0.0187017752942 1.0 1.0


ok they performed pretty well. perfect roc and accuracy and a loss of 0.01870 on training. this coudl be overfit but i guess well find out soon. i fell comfrtable using one submission on this, more robust than my last 2 submissions. OR we could try adding convolutional features to the ensemble, combined w/ xgboost. hmmm.... lets try that shit, see if it makes loss worse or better. now i want to predict the test set and submit those preds.

In [29]:
preds_df_test = pd.DataFrame([], columns=['name'])
preds_df_test['name'] = test_set['name']

for model_name in keep_models:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_inceptionv3(test_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_inceptionv3(test_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_inceptionv3(test_img_450.astype(np.float32))
    preds_df_test[model_name] = pd.Series(model.predict(proc_img).flatten())
    delete_model(model)
preds_df_test.head()

invasive_incepv3_aug10_kfolds_299x299_0.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


invasive_xcep_aug11_kfolds_224x224_0.model
invasive_xcep_aug11_kfolds_224x224_3.model
invasive_xcep_aug11_kfolds_299x299_0.model
invasive_xcep_aug11_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_299x299_2.model
invasive_xcep_aug11_kfolds_299x299_3.model
invasive_xcep_aug11_kfolds_299x299_4.model
invasive_xcep_aug13_pseudo_224x224_0.model
invasive_xcep_aug13_pseudo_224x224_1.model
invasive_xcep_aug13_pseudo_224x224_2.model
invasive_xcep_aug13_pseudo_224x224_3.model
invasive_xcep_aug13_pseudo_224x224_4.model
invasive_xcep_aug14_pseudo_ft_299x299_0.model
invasive_xcep_aug14_pseudo_ft_299x299_1.model
invasive_xcep_aug14_pseudo_ft_299x299_2.model
invasive_xcep_aug14_pseudo_ft_299x299_3.model
invasive_xcep_aug14_pseudo_ft_299x299_4.model
invasive_incepv3_aug11_kfolds_224x224_1.model
invasive_incepv3_aug11_kfolds_224x224_2.model
invasive_incepv3_aug11_kfolds_224x224_3.model
invasive_incepv3_aug11_kfolds_450x450_3.model
invasive_incepv3_aug11_kfolds_450x450_4.model
invasive_incepv3_aug13_p

,name,invasive_incepv3_aug10_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_224x224_0.model,invasive_xcep_aug11_kfolds_224x224_3.model,invasive_xcep_aug11_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_3.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_xcep_aug13_pseudo_224x224_0.model,...,invasive_incepv3_aug13_pseudo_224x224_0.model,invasive_incepv3_aug13_pseudo_224x224_1.model,invasive_incepv3_aug13_pseudo_224x224_2.model,invasive_incepv3_aug13_pseudo_224x224_3.model,invasive_incepv3_aug13_pseudo_224x224_4.model,invasive_incepv3_aug13_pseudo_299x299_0.model,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_2.model,invasive_incepv3_aug13_pseudo_299x299_3.model,invasive_incepv3_aug13_pseudo_299x299_4.model
0,1,0.999220,0.999832,0.999998,1.000000,1.000000,0.999998,1.000000,1.000000,0.999370,...,0.995812,0.990820,0.995529,0.990677,0.996971,0.992153,0.997154,0.990139,0.991377,0.988569
1,2,0.006064,0.292350,0.011755,0.087101,0.000298,0.003632,0.012436,0.011816,0.040062,...,0.012368,0.034217,0.030731,0.032573,0.046906,0.010195,0.025843,0.056894,0.051597,0.026488
2,3,0.011985,0.047080,0.055718,0.012156,0.014143,0.000052,0.001061,0.007512,0.048727,...,0.305156,0.084924,0.042472,0.016454,0.090059,0.051287,0.097718,0.122369,0.107176,0.085269
3,4,0.017386,0.000072,0.000183,0.000373,0.005312,0.000028,0.001051,0.000032,0.002739,...,0.042333,0.088001,0.030367,0.019637,0.034847,0.017609,0.025839,0.017977,0.035340,0.034613
4,5,0.978103,0.975208,0.989112,0.999627,0.999979,0.999182,0.998613,0.999819,0.966815,...,0.993808,0.991267,0.991358,0.980899,0.987864,0.971801,0.988640,0.980422,0.966536,0.960517


In [30]:
subm = pd.DataFrame([], columns=['name', 'invasive'])
subm['invasive'] = preds_df_test[[c for c in preds_df_test.columns if c in keep_models]].mean(axis=1)
subm['name'] =  preds_df_test['name']
subm.head()

,name,invasive
0,1,0.995828
1,2,0.032801
2,3,0.062109
3,4,0.016720
4,5,0.986963


In [31]:
subm.to_csv(os.path.join(path, 'results', 'subm_aug15_1.gz'), index=False, compression='gzip')

alright that scored 0.9513 (same as my current score) so it confirms my theroy about handpicking but this way is better than hand picking. (more robust)

now we will make an xgboost model that takes the convolutional features from each model's output. we will then see if adding this model improves things. maybe we should try that on the last submission? yea because idk if it will work, as i havent had success so far. so that should be the last thing i try.

In [42]:
xgb_train_preds, xgb_valid_preds, xgb_train_labels, xgb_valid_labels = train_test_split(preds_df_train.values, train_labels, test_size=0.2, random_state=64)
xgb = XGBClassifier()
_ = xgb.fit(xgb_train_preds, xgb_train_labels)

In [43]:
forest_preds_train = xgb.predict_proba(xgb_train_preds)
forest_preds_train = forest_preds_train[:,1] # get the likelihood of the 1 class (0 is at index 0)
forest_preds_valid = xgb.predict_proba(xgb_valid_preds)
forest_preds_valid = forest_preds_valid[:,1] # get the likelihood of the 1 class (0 is at index 0)

print('forest train loss:')
print('{}'.format(log_loss(xgb_train_labels, forest_preds_train)))
print('forest train aroc:')
print('{}'.format(roc_auc_score(xgb_train_labels, forest_preds_train)))
print('forest train accuracy:')
print('{}'.format(accuracy_score(xgb_train_labels, np.around(forest_preds_train))))

print('forest valid loss:')
print('{}'.format(log_loss(xgb_valid_labels, forest_preds_valid)))
print('forest valid aroc:')
print('{}'.format(roc_auc_score(xgb_valid_labels, forest_preds_valid)))
print('forest valid accuracy:')
print('{}'.format(accuracy_score(xgb_valid_labels, np.around(forest_preds_valid))))
print('\n')

forest train loss:
0.001075071887841574
forest train aroc:
1.0
forest train accuracy:
1.0
forest valid loss:
0.0010916514148172761
forest valid aroc:
1.0
forest valid accuracy:
1.0




the accuracy is actually perfect here now. hmmm... should i use xgb for my rinal sumission or should i add vgg/restnet. im leaning towards resnet+vgg, xgb hasnt worked in the past... although i've neve got it to move its validation loss like this (i did have perfect accuracy on validation before but the loss was never this low/this consistent on validation. ill examine both and see which one imakes more sense to submit. we cant fit xgboost w/o validation data.

In [54]:
preds_df_train = pd.DataFrame([], columns=['name'])
preds_df_train['name'] = train_labels

resnet_vgg_model_files = [
    'invasive_resnet50_aug15_pseudo_224x224_0.model',
    'invasive_resnet50_aug15_pseudo_224x224_1.model',
    'invasive_resnet50_aug15_pseudo_224x224_2.model',
    'invasive_resnet50_aug15_pseudo_299x299_0.model',
    'invasive_resnet50_aug15_pseudo_299x299_1.model',
    'invasive_vgg16_aug15_pseudo_224x224_0.model'
]

for model_name in resnet_vgg_model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_resnet_vgg(train_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_resnet_vgg(train_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_resnet_vgg(train_img_450.astype(np.float32))
    preds_df_train[model_name] = pd.Series(model.predict(proc_img).flatten())
    delete_model(model)
preds_df_train.head()

invasive_resnet50_aug15_pseudo_224x224_0.model
invasive_resnet50_aug15_pseudo_224x224_1.model
invasive_resnet50_aug15_pseudo_299x299_0.model
invasive_vgg16_aug15_pseudo_224x224_0.model


,name,invasive_resnet50_aug15_pseudo_224x224_0.model,invasive_resnet50_aug15_pseudo_224x224_1.model,invasive_resnet50_aug15_pseudo_299x299_0.model,invasive_vgg16_aug15_pseudo_224x224_0.model
0,0,0.002091,0.001616,1.090576e-03,0.015221
1,0,0.002525,0.001423,9.392508e-07,0.041529
2,1,0.998409,0.999330,9.980170e-01,0.993860
3,0,0.012259,0.020136,7.734579e-03,0.180848
4,1,0.999394,0.999771,9.998239e-01,0.999198


In [55]:
keep_models = []
all_loss = 0
for col in preds_df_train.columns:
    train_preds = pd.DataFrame([], columns=['name', 'invasive'])
    if col == 'name':
        exclusion_list = []
    else:
        exclusion_list = [col]
        
    train_preds['invasive'] = preds_df_train[[c for c in preds_df_train.columns if not c in exclusion_list]].mean(axis=1)
    train_preds['name'] =  preds_df_train['name']
    
    # calculate the loss, roc, and accuracy 
    # without thes model
    loss_without = log_loss(train_labels, train_preds['invasive'])
    roc_without = roc_auc_score(train_labels, train_preds['invasive'])
    acc_without = accuracy_score(train_labels, np.around(train_preds['invasive']))
    
    print('without {}'.format(col))
    print(loss_without, roc_without, acc_without)
    
    if col == 'name':
        all_loss = loss_without
        
    print('is lower than all?')
    print(loss_without >= all_loss )
    
    # if loss withoht this model is higher than
    # the loss with all models keep the model
    # we want to aboid loss going higher.
    if (loss_without >= all_loss) and (col != 'name'):
        keep_models.append(col)

    print('\n')

without name
0.0112750405324 1.0 1.0
is lower than all?
True


without invasive_resnet50_aug15_pseudo_224x224_0.model
0.0120772461163 1.0 1.0
is lower than all?
True


without invasive_resnet50_aug15_pseudo_224x224_1.model
0.0115313407354 1.0 1.0
is lower than all?
True


without invasive_resnet50_aug15_pseudo_299x299_0.model
0.0119099946147 1.0 1.0
is lower than all?
True


without invasive_vgg16_aug15_pseudo_224x224_0.model
0.00695187378351 1.0 1.0
is lower than all?
False




ok so we want to ditch vgg, keep resnets. resnets improve the avg, vgg kills it. this doestn really work without al lthe models. because the first value is the avreag of all of them. so it just means vgg was worse than the avg. if we keep repeating this binary process well be left with one model. vgg had a bad loss so its best to ditch it.

In [6]:
final_model_files = [
  'invasive_incepv3_aug10_kfolds_299x299_0.model',
  'invasive_xcep_aug11_kfolds_224x224_0.model',
  'invasive_xcep_aug11_kfolds_224x224_3.model',
  'invasive_xcep_aug11_kfolds_299x299_0.model',
  'invasive_xcep_aug11_kfolds_299x299_1.model',
  'invasive_xcep_aug11_kfolds_299x299_2.model',
  'invasive_xcep_aug11_kfolds_299x299_3.model',
  'invasive_xcep_aug11_kfolds_299x299_4.model',
  'invasive_xcep_aug13_pseudo_224x224_0.model',
  'invasive_xcep_aug13_pseudo_224x224_1.model',
  'invasive_xcep_aug13_pseudo_224x224_2.model',
  'invasive_xcep_aug13_pseudo_224x224_3.model',
  'invasive_xcep_aug13_pseudo_224x224_4.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_0.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_3.model',
  'invasive_xcep_aug14_pseudo_ft_299x299_4.model',
  'invasive_incepv3_aug11_kfolds_224x224_1.model',
  'invasive_incepv3_aug11_kfolds_224x224_2.model',
  'invasive_incepv3_aug11_kfolds_224x224_3.model',
  'invasive_incepv3_aug11_kfolds_450x450_3.model',
  'invasive_incepv3_aug11_kfolds_450x450_4.model',
  'invasive_incepv3_aug13_pseudo_224x224_0.model',
  'invasive_incepv3_aug13_pseudo_224x224_1.model',
  'invasive_incepv3_aug13_pseudo_224x224_2.model',
  'invasive_incepv3_aug13_pseudo_224x224_3.model',
  'invasive_incepv3_aug13_pseudo_224x224_4.model',
  'invasive_incepv3_aug13_pseudo_299x299_0.model',
  'invasive_incepv3_aug13_pseudo_299x299_1.model',
  'invasive_incepv3_aug13_pseudo_299x299_2.model',
  'invasive_incepv3_aug13_pseudo_299x299_3.model',
  'invasive_incepv3_aug13_pseudo_299x299_4.model'
]

preds_df_test = pd.DataFrame([], columns=['name'])
preds_df_test['name'] = test_set['name']

for model_name in final_model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_incep_xcep(test_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_incep_xcep(test_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_incep_xcep(test_img_450.astype(np.float32))
    preds_df_test[model_name] = pd.Series(model.predict(proc_img).flatten())
    print(preds_df_test.shape)
    delete_model(model)
preds_df_test.head()

invasive_incepv3_aug10_kfolds_299x299_0.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


(1531, 2)
invasive_xcep_aug11_kfolds_224x224_0.model
(1531, 3)
invasive_xcep_aug11_kfolds_224x224_3.model
(1531, 4)
invasive_xcep_aug11_kfolds_299x299_0.model
(1531, 5)
invasive_xcep_aug11_kfolds_299x299_1.model
(1531, 6)
invasive_xcep_aug11_kfolds_299x299_2.model
(1531, 7)
invasive_xcep_aug11_kfolds_299x299_3.model
(1531, 8)
invasive_xcep_aug11_kfolds_299x299_4.model
(1531, 9)
invasive_xcep_aug13_pseudo_224x224_0.model
(1531, 10)
invasive_xcep_aug13_pseudo_224x224_1.model
(1531, 11)
invasive_xcep_aug13_pseudo_224x224_2.model
(1531, 12)
invasive_xcep_aug13_pseudo_224x224_3.model
(1531, 13)
invasive_xcep_aug13_pseudo_224x224_4.model
(1531, 14)
invasive_xcep_aug14_pseudo_ft_299x299_0.model
(1531, 15)
invasive_xcep_aug14_pseudo_ft_299x299_1.model
(1531, 16)
invasive_xcep_aug14_pseudo_ft_299x299_2.model
(1531, 17)
invasive_xcep_aug14_pseudo_ft_299x299_3.model
(1531, 18)
invasive_xcep_aug14_pseudo_ft_299x299_4.model
(1531, 19)
invasive_incepv3_aug11_kfolds_224x224_1.model
(1531, 20)
invasiv

,name,invasive_incepv3_aug10_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_224x224_0.model,invasive_xcep_aug11_kfolds_224x224_3.model,invasive_xcep_aug11_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_3.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_xcep_aug13_pseudo_224x224_0.model,...,invasive_incepv3_aug13_pseudo_224x224_0.model,invasive_incepv3_aug13_pseudo_224x224_1.model,invasive_incepv3_aug13_pseudo_224x224_2.model,invasive_incepv3_aug13_pseudo_224x224_3.model,invasive_incepv3_aug13_pseudo_224x224_4.model,invasive_incepv3_aug13_pseudo_299x299_0.model,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_2.model,invasive_incepv3_aug13_pseudo_299x299_3.model,invasive_incepv3_aug13_pseudo_299x299_4.model
0,1,0.999220,0.999832,0.999998,1.000000,1.000000,0.999998,1.000000,1.000000,0.999370,...,0.995812,0.990820,0.995529,0.990677,0.996971,0.992153,0.997154,0.990139,0.991377,0.988569
1,2,0.006064,0.292350,0.011755,0.087101,0.000298,0.003632,0.012436,0.011816,0.040062,...,0.012368,0.034217,0.030731,0.032573,0.046906,0.010195,0.025843,0.056894,0.051597,0.026488
2,3,0.011985,0.047080,0.055718,0.012156,0.014143,0.000052,0.001061,0.007512,0.048727,...,0.305156,0.084924,0.042472,0.016454,0.090059,0.051287,0.097718,0.122369,0.107176,0.085269
3,4,0.017386,0.000072,0.000183,0.000373,0.005312,0.000028,0.001051,0.000032,0.002739,...,0.042333,0.088001,0.030367,0.019637,0.034847,0.017609,0.025839,0.017977,0.035340,0.034613
4,5,0.978103,0.975208,0.989112,0.999627,0.999979,0.999182,0.998613,0.999819,0.966815,...,0.993808,0.991267,0.991358,0.980899,0.987864,0.971801,0.988640,0.980422,0.966536,0.960517


In [7]:
more_model_files = [  
  'invasive_resnet50_aug15_pseudo_224x224_0.model',
  'invasive_resnet50_aug15_pseudo_224x224_1.model',
  'invasive_resnet50_aug15_pseudo_224x224_2.model',
  'invasive_resnet50_aug15_pseudo_299x299_0.model',
  'invasive_resnet_aug15_kfolds_224x224_0.model',
  'invasive_resnet_aug15_kfolds_224x224_1.model'
]

for model_name in more_model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_resnet_vgg(test_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_resnet_vgg(test_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_resnet_vgg(test_img_450.astype(np.float32))
    preds_df_test[model_name] = pd.Series(model.predict(proc_img).flatten())
    print(preds_df_test.shape)
    delete_model(model)
preds_df_test.head()

invasive_resnet50_aug15_pseudo_224x224_0.model
(1531, 35)
invasive_resnet50_aug15_pseudo_224x224_1.model
(1531, 36)
invasive_resnet50_aug15_pseudo_224x224_2.model
(1531, 37)
invasive_resnet50_aug15_pseudo_299x299_0.model
(1531, 38)
invasive_resnet_aug15_kfolds_224x224_0.model
(1531, 39)
invasive_resnet_aug15_kfolds_224x224_1.model
(1531, 40)


,name,invasive_incepv3_aug10_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_224x224_0.model,invasive_xcep_aug11_kfolds_224x224_3.model,invasive_xcep_aug11_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_3.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_xcep_aug13_pseudo_224x224_0.model,...,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_2.model,invasive_incepv3_aug13_pseudo_299x299_3.model,invasive_incepv3_aug13_pseudo_299x299_4.model,invasive_resnet50_aug15_pseudo_224x224_0.model,invasive_resnet50_aug15_pseudo_224x224_1.model,invasive_resnet50_aug15_pseudo_224x224_2.model,invasive_resnet50_aug15_pseudo_299x299_0.model,invasive_resnet_aug15_kfolds_224x224_0.model,invasive_resnet_aug15_kfolds_224x224_1.model
0,1,0.999220,0.999832,0.999998,1.000000,1.000000,0.999998,1.000000,1.000000,0.999370,...,0.997154,0.990139,0.991377,0.988569,0.994815,0.991178,0.993879,0.996044,9.998395e-01,0.999653
1,2,0.006064,0.292350,0.011755,0.087101,0.000298,0.003632,0.012436,0.011816,0.040062,...,0.025843,0.056894,0.051597,0.026488,0.007057,0.026384,0.007962,0.012463,8.163955e-09,0.000024
2,3,0.011985,0.047080,0.055718,0.012156,0.014143,0.000052,0.001061,0.007512,0.048727,...,0.097718,0.122369,0.107176,0.085269,0.032197,0.006122,0.028338,0.011865,1.050557e-03,0.000830
3,4,0.017386,0.000072,0.000183,0.000373,0.005312,0.000028,0.001051,0.000032,0.002739,...,0.025839,0.017977,0.035340,0.034613,0.006470,0.012695,0.016014,0.010219,1.422963e-06,0.000381
4,5,0.978103,0.975208,0.989112,0.999627,0.999979,0.999182,0.998613,0.999819,0.966815,...,0.988640,0.980422,0.966536,0.960517,0.996808,0.949328,0.992435,0.989750,9.998783e-01,0.990446


In [8]:
subm = pd.DataFrame([], columns=['name', 'invasive'])
subm['invasive'] = preds_df_test[[c for c in preds_df_test.columns if c in final_model_files+more_model_files]].mean(axis=1)
subm['name'] =  preds_df_test['name']
subm.head()

,name,invasive
0,1,0.995839
1,2,0.029136
2,3,0.054615
3,4,0.015321
4,5,0.986883


In [9]:
subm.to_csv(os.path.join(path, 'results', 'subm_aug15_2.gz'), index=False, compression='gzip')

# awful

In [70]:
# input 224 because running outta time
resnet = ResNet50(input_shape=(224,224,3), weights='imagenet', include_top=False, pooling=None)
inception = InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False, pooling=None)
xception = Xception(input_shape=(224,224,3), weights='imagenet', include_top=False, pooling=None)

resnet_feat = resnet.predict(train_img_224)
inception_feat = inception.predict(train_img_224)
xception_feat = xception.predict(train_img_224)

xgb_train_preds, xgb_valid_preds, xgb_train_labels, xgb_valid_labels = train_test_split(np.hstack([resnet_feat.reshape(2295,-1), inception_feat.reshape(2295,-1), xception_feat.reshape(2295,-1)]), train_labels, test_size=0.2, random_state=64)
resnet_feat.reshape(2295,-1).shape, inception_feat.reshape(2295,-1).shape, xception_feat.reshape(2295,-1).shape

((2295, 2048), (2295, 51200), (2295, 100352))

In [71]:
xgb = RandomForestClassifier()
_ = xgb.fit(xgb_train_preds, xgb_train_labels)

In [72]:
forest_preds_train = xgb.predict_proba(xgb_train_preds)
forest_preds_train = forest_preds_train[:,1] # get the likelihood of the 1 class (0 is at index 0)
forest_preds_valid = xgb.predict_proba(xgb_valid_preds)
forest_preds_valid = forest_preds_valid[:,1] # get the likelihood of the 1 class (0 is at index 0)

print('forest train loss:')
print('{}'.format(log_loss(xgb_train_labels, forest_preds_train)))
print('forest train aroc:')
print('{}'.format(roc_auc_score(xgb_train_labels, forest_preds_train)))
print('forest train accuracy:')
print('{}'.format(accuracy_score(xgb_train_labels, np.around(forest_preds_train))))

print('forest valid loss:')
print('{}'.format(log_loss(xgb_valid_labels, forest_preds_valid)))
print('forest valid aroc:')
print('{}'.format(roc_auc_score(xgb_valid_labels, forest_preds_valid)))
print('forest valid accuracy:')
print('{}'.format(accuracy_score(xgb_valid_labels, np.around(forest_preds_valid))))
print('\n')

forest train loss:
0.11765916936449776
forest train aroc:
0.9998267498267499
forest train accuracy:
0.9950980392156863
forest valid loss:
0.4283605221507064
forest valid aroc:
0.9305218808378711
forest valid accuracy:
0.840958605664488




ok this is awful in its form as is. random forest or xgboost on just pretrained features are probabl ya waste of time this late in.